In [ ]:
def calculate_energy_requirement_for_dewatering_per_day(
    self, input_type: str = "default"
):
    """Energy required by the de-watering unit in kWh/day."""
    # todo check with Walter/ Arnold
    wet_product = (
        # would suggest renaming: water_removed_by_mechanical_dewatering
        self.calculate_total_wet_product_usage_per_hour(input_type=input_type) * 24
        #Walter: I assume this is total water removed by Larox (which should be: Dry output = mass-flow EMU out * (Dry solids content out))

        # Water removed = output mass EMU (total) *(1-DS EMU Out/ DS Larox Out)
        # how do we trade off condensed formula's to readible formulas (or break them down into intuitively understandable steps?)
        # DS output EMU = function of DS input EMU, purity, carbonation rate etc, or it is simply an input parameter for now).
        # DS output Larox is a an input parameter (default 75%)
        # These numbers could also follow from a 'Mass balance' Function, where we calculate the mass-balance over all proces steps.... and even detail out into constituents, like Forsterite, CO2, CRAP, MgCO3 etc. This might be a code-module which is also very usefull outside TELCA...
    )
    power_efficiency = get_value(
        self.config["de-watering"]["design_default_type"]["power_efficiency"],
        input_type,
        #Walter: I assume this factor is used on top of the literature value of 42MJ/t water removed. A simple parameter to fit Literature value to what we will measure in the field.
        #Percentages should alreay be 0-1 on code level (in my opinion)
    )
    mechanical_dewatering_energy_usage_per_tonne = get_value(
        #would suggest renaming to: mechanical_dewatering_energy_usage_per_ton_water
        self.config["de-watering"]["design_default_type"]["energy_usage_per_tonne"],
        input_type,
    )
    electricity_usage_per_day = (
        mechanical_dewatering_energy_usage_per_tonne
        * power_efficiency # normally you would divide by effeciency (end result should go up if effeciency is low)
        / 100 # to me this should not be done on code level, but on parameter read level
        * wet_product
        / 1000 #convert from kg to ton?
        # we should eiter have a code level wide stanard for units, or include unit in variable name.
    )
    return electricity_usage_per_day

In [ ]:
def calculate_energy_requirement_for_drying_per_day(
    self, input_type: str = "default"
):
    """Energy required by the drying unit in kWh/day."""
    # todo check with Walter/ Arnold
    dry_product = (
        #would suggest renaming to water_removed_by_thermal_drying
        self.calculate_total_dry_product_usage_per_hour(input_type=input_type) * 24
        # Water removed = output mass larox (total) *(1-DS larox Out/ DS dryer Out) -> this typically be per hour (whole mass balance is typilly done per hour, and then the numbers would be ...rate)
        # DS dryer out should be an input parameter
        # conceptually: how generic/specific should variable names be (larox vs mechancial dewatering, dryer vs flash-dryer vs thermal drying). How long do we want variable names to become....

    )
    power_efficiency = get_value(
        self.config["drying"]["design_default_type"]["power_efficiency"], input_type
        #number between 0 and 1?
    )
    thermal_drying_energy_usage_per_tonne = get_value(
        #suggest renaming to  thermal_drying_energy_usage_per_ton_water
        self.config["drying"]["design_default_type"]["energy_usage_per_tonne"],
        input_type,
        # This should be a number per ton of water evaporated. (630kwh/t of water evaporated)
    )
    electricity_usage_per_day = (
        thermal_drying_energy_usage_per_tonne
        * power_efficiency # change to devision
        / 100 # remove this, and handle at parameter input level
        * dry_product
        / 1000
    )
    return electricity_usage_per_day

In [ ]:
def get_model_inputs(self, input_type: str = "default"):
    """Calculate the fundamental inputs of the system."""
    co2_storage_temperature = get_value(
        self.config["reactor"]["design_default_type"]["co2_storage_temperature"],
        input_type, #in oC
    )
    co2_storage_pressure = get_value(
        self.config["reactor"]["design_default_type"]["co2_storage_pressure"],
        input_type, # in barg
    )
    co2_density = CP.PropsSI(
        "D",
        "T",
        co2_storage_temperature + 273.15,
        "P",
        (co2_storage_pressure + 1.01325) * 100000,
        "CO2",
    )
    co2_mass_flow_rate = get_value(
        self.config["reactor"]["design_default_type"]["co2_feed_rate"], input_type #this should come from stochiometric calculation, not input
        # this should come from a properly setup mass-balance module... It is basically a table with all steps in the proces, containing the properties of the flows. It is very easy to do in Excel, I wouldn't know how to set it up in Python, but it might be just as easy....
    )  # todo correct with Louk/ Walter
    co2_discharge_pressure = (
        get_value(
            self.config["reactor"]["design_default_type"]["co2_inflow_pressure"],
            input_type,
        )
        + 6.0 #?
    )
    slurry_discharge_pressure = (
        get_value(
            self.config["reactor"]["design_default_type"]["co2_inflow_pressure"],
            input_type,
        )
        + 3.8 #?
    )
    co2_evaporator_temperature = get_value(
        self.config["reactor"]["design_default_type"]["co2_evaporator_temperature"],
        input_type,
    )
    slurry_feed_rate = get_value(
        self.config["reactor"]["design_default_type"]["slurry_feed_rate"],
        input_type,
    )
    slurry_discharge_rate = slurry_feed_rate  # todo check with Louk/ Walter
    slurry_input_pressure = get_value(
        self.config["reactor"]["design_default_type"]["slurry_input_pressure"],
        input_type,
    )
    slurry_discharge_average_heat_capacity = 3.49  # todo update with Louk/ Walter
    slurry_discharge_temperature = 36.0  # todo update with Louk/ Walter
    slurry_heater_to_reactor_average_heat_capacity = (
        3.49  # todo update with Louk/ Walter
    )
    slurry_heater_to_reactor_temperature = 36.0  # todo update with Louk/ Walter
    circulation_water_outlet_flow_rate = 175.0  # todo update with Louk/ Walter
    circulation_water_outlet_average_heat_capacity = (
        4.18  # todo update with Louk/ Walter
    )
    circulation_water_outlet_temperature = 175.0  # todo update with Louk/ Walter
    circulation_water_cooled_temperature = 36.0  # todo update with Louk/ Walter
    product_cooler_mass_flow_rate = 0.0  # todo update with Louk/ Walter
    product_cooler_average_heat_capacity = 4.18  # todo update with Louk/ Walter
    reactor_to_product_cooler_temperature = 36.0  # todo update with Louk/ Walter
    product_cooler_to_outlet_temperature = 36.0  # todo update with Louk/ Walter

    return {
        "co2_storage_temperature": co2_storage_temperature,
        "co2_storage_pressure": co2_storage_pressure,
        "co2_density": co2_density,
        "co2_mass_flow_rate": co2_mass_flow_rate,
        "co2_discharge_pressure": co2_discharge_pressure,
        "co2_evaporator_temperature": co2_evaporator_temperature,
        "slurry_discharge_pressure": slurry_discharge_pressure,
        "slurry_feed_rate": slurry_feed_rate,
        "slurry_discharge_rate": slurry_discharge_rate,
        "slurry_input_pressure": slurry_input_pressure,
        "slurry_discharge_average_heat_capacity": slurry_discharge_average_heat_capacity,
        "slurry_heater_to_reactor_average_heat_capacity": slurry_heater_to_reactor_average_heat_capacity,
        "slurry_heater_to_reactor_temperature": slurry_heater_to_reactor_temperature,
        "slurry_discharge_temperature": slurry_discharge_temperature,
        "circulation_water_outlet_flow_rate": circulation_water_outlet_flow_rate,
        "circulation_water_outlet_average_heat_capacity": circulation_water_outlet_average_heat_capacity,
        "circulation_water_outlet_temperature": circulation_water_outlet_temperature,
        "circulation_water_cooled_temperature": circulation_water_cooled_temperature,
        "product_cooler_mass_flow_rate": product_cooler_mass_flow_rate,
        "product_cooler_average_heat_capacity": product_cooler_average_heat_capacity,
        "reactor_to_product_cooler_temperature": reactor_to_product_cooler_temperature,
        "product_cooler_to_outlet_temperature": product_cooler_to_outlet_temperature,
    }

In [ ]:
def calculate_co2_pump(self, input_type: str = "default"):
    """Energy usage by CO2 pumped into the reactor in kWh."""
    co2_storage_pressure = self.get_model_inputs(input_type)["co2_storage_pressure"]
    co2_density = self.get_model_inputs(input_type)["co2_density"]
    co2_mass_flow_rate = self.get_model_inputs(input_type)["co2_mass_flow_rate"]
    co2_volume_flow_rate = co2_mass_flow_rate / co2_density
    co2_discharge_pressure = self.get_model_inputs(input_type)[
        "co2_discharge_pressure"
    ]
    co2_pump_energy = (
        co2_volume_flow_rate
        * (co2_discharge_pressure - co2_storage_pressure)
        / 36
        / 0.5
    )
    return co2_pump_energy

In [ ]:
def calculate_co2_heater(self, input_type: str = "default"):
    """Energy usage by CO2 heater in kWh."""
    co2_evaporator_temperature = self.get_model_inputs(input_type)[
        "co2_evaporator_temperature"
    ]
    co2_discharge_pressure = self.get_model_inputs(input_type)[
        "co2_discharge_pressure"
    ]
    co2_storage_temperature = self.get_model_inputs(input_type)[
        "co2_storage_temperature"
    ]
    co2_storage_pressure = self.get_model_inputs(input_type)["co2_storage_pressure"]
    co2_pump_energy = self.calculate_co2_pump(input_type)
    co2_mass_flow_rate = self.get_model_inputs(input_type)["co2_mass_flow_rate"]
    co2_pump_discharge_evaporator_temperature = (
        CP.PropsSI(
            "T",
            "H",
            CP.PropsSI(
                "H",
                "T",
                co2_storage_temperature + 273.15,
                "P",
                (co2_storage_pressure + 1.01325) * 100000,
                "CO2",
            )
            + co2_pump_energy * 1000 / (co2_mass_flow_rate / 3600),
            "P",
            (co2_discharge_pressure + 1.01325) * 100000,
            "CO2",
        )
        - 273.15
    )  # todo check calculations with Louk/ Walter
    co2_heater_energy = (
        (
            CP.PropsSI(
                "H",
                "T",
                co2_evaporator_temperature + 273.15,
                "P",
                (co2_discharge_pressure + 1.01325) * 100000,
                "CO2",
            )
            - CP.PropsSI(
                "H",
                "T",
                co2_pump_discharge_evaporator_temperature + 273.15,
                "P",
                (co2_discharge_pressure + 1.01325) * 100000,
                "CO2",
            )
        )
        * co2_mass_flow_rate
        / 3600
        / 1000
    )  # todo check calculations with Louk/ Walter
    return co2_heater_energy

In [ ]:
def calculate_power_imported(self, input_type: str = "default"):
    """Energy imported in kWh."""
    # todo check formulae with Louk/ Walter
    slurry_feed_rate = self.get_model_inputs(input_type)["slurry_feed_rate"]
    slurry_discharge_pressure = self.get_model_inputs(input_type)[
        "slurry_discharge_pressure"
    ]
    slurry_input_pressure = self.get_model_inputs(input_type)[
        "slurry_input_pressure"
    ]
    F100 = (
        slurry_feed_rate
        * (slurry_discharge_pressure - slurry_input_pressure)
        / 36
        / (0.897 * 0.911)
    )

    slurry_discharge_rate = self.get_model_inputs(input_type)[
        "slurry_discharge_rate"
    ]
    slurry_discharge_average_heat_capacity = self.get_model_inputs(input_type)[
        "slurry_discharge_average_heat_capacity"
    ]
    slurry_heater_to_reactor_average_heat_capacity = self.get_model_inputs(
        input_type
    )["slurry_heater_to_reactor_average_heat_capacity"]
    slurry_heater_to_reactor_temperature = self.get_model_inputs(input_type)[
        "slurry_heater_to_reactor_temperature"
    ]
    slurry_discharge_temperature = self.get_model_inputs(input_type)[
        "slurry_discharge_temperature"
    ]
    H101 = (
        slurry_discharge_rate
        / 3600
        * (
            slurry_discharge_average_heat_capacity
            + slurry_heater_to_reactor_average_heat_capacity
        )
        / 2
        * (slurry_heater_to_reactor_temperature - slurry_discharge_temperature)
    )

    circulation_water_outlet_flow_rate = self.get_model_inputs(input_type)[
        "circulation_water_outlet_flow_rate"
    ]
    circulation_water_outlet_average_heat_capacity = self.get_model_inputs(
        input_type
    )["circulation_water_outlet_average_heat_capacity"]
    circulation_water_outlet_temperature = self.get_model_inputs(input_type)[
        "circulation_water_outlet_temperature"
    ]
    circulation_water_cooled_temperature = self.get_model_inputs(input_type)[
        "circulation_water_cooled_temperature"
    ]
    J102 = (
        circulation_water_outlet_flow_rate
        / 3600
        * circulation_water_outlet_average_heat_capacity
        * (
            circulation_water_outlet_temperature
            - circulation_water_cooled_temperature
        )
    )

    product_cooler_mass_flow_rate = self.get_model_inputs(input_type)[
        "product_cooler_mass_flow_rate"
    ]
    product_cooler_average_heat_capacity = self.get_model_inputs(input_type)[
        "product_cooler_average_heat_capacity"
    ]
    reactor_to_product_cooler_temperature = self.get_model_inputs(input_type)[
        "reactor_to_product_cooler_temperature"
    ]
    product_cooler_to_outlet_temperature = self.get_model_inputs(input_type)[
        "product_cooler_to_outlet_temperature"
    ]
    Y100 = (
        product_cooler_mass_flow_rate
        / 3600
        * product_cooler_average_heat_capacity
        * (
            reactor_to_product_cooler_temperature
            - product_cooler_to_outlet_temperature
        )
    )

    instrument_air = get_value(
        self.config["reactor"]["design_default_type"]["instrument_air"], input_type
    )

    steam_consumption = get_value(
        self.config["reactor"]["design_default_type"]["steam_consumption"],
        input_type,
    )

    power_imported = F100 + H101 + J102 + Y100 + instrument_air + steam_consumption
    return power_imported

In [ ]:
def calculate_energy_requirement_for_reactor_per_day(
    self, input_type: str = "default"
):
    """Energy required by the reactor in kWh/day."""
    # todo check with Walter/ Arnold
    co2_pump_energy = self.calculate_co2_pump(input_type=input_type)
    co2_heater_energy = self.calculate_co2_heater(input_type=input_type)
    power_imported = self.calculate_power_imported(input_type=input_type)
    electricity_usage_per_day = (
        co2_pump_energy + co2_heater_energy + power_imported
    ) * 24
    return electricity_usage_per_day

In [ ]:
def calculate_heat_generated_by_reaction(self):
    """Calculate the heat generated by the reaction in kWh/day."""
    # todo check with Walter/ Arnold
    # todo remove recovered energy from reaction from energy requirement per day
    temperature_cooler_outlet = 30.0
    temperature_dryer_output = 157.8
    mass_flow_cooler_loop = 0.0
    cooler_flow_mass_heat_capacity = 4.18
    if mass_flow_cooler_loop > 0:
        heat_generated_by_reaction = (
            mass_flow_cooler_loop
            / 3600
            * cooler_flow_mass_heat_capacity
            * (temperature_cooler_outlet - temperature_dryer_output)
        )
    else:
        heat_generated_by_reaction = 0
    return heat_generated_by_reaction

In [ ]:
def calculate_unavoidable_co2(self, input_type: str = "default"):
    """Calculate the unavoidable CO2 emissions in kg/day."""
    # todo check with Walter/ Arnold

    # reactor =('H&M balance'!P31+'H&M balance'!X42)*24

    co2_mass_flow_rate = self.get_model_inputs(input_type)["co2_mass_flow_rate"]
    co2_emission_factor = (
        get_value(
            self.config["reactor"]["design_default_type"]["co2_emission_factor"],
            input_type,
        )
        / 100
    )
    co2_purged_per_hour = co2_mass_flow_rate * co2_emission_factor + 0.000001
    # X42 = O24-P31-P32-X43-X53-X65-X71-X59

    unavoidable_co2 = (
        co2_purged_per_hour * 24
    )  # +(*X42 * 24) todo check with Walter/ Louk

    return unavoidable_co2